In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pickle


In [22]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [23]:
## DATA PREPROCESSING
data = data.drop(columns=["RowNumber", "CustomerId", "Surname"], axis =1)


In [24]:
## ENCODE THE CATEGORICAL DATA
l = LabelEncoder()
data["Gender"] = l.fit_transform(data["Gender"])


In [25]:
## ONE HOT ENCODING FOR GEOGRAPHY
o = OneHotEncoder()
geo_encoded = o.fit_transform(data[["Geography"]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=o.get_feature_names_out(["Geography"]))
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [26]:
# CONCATENATE THE ENCODED DATA WITH THE ORIGINAL DATA
data = pd.concat([data.drop(["Geography"], axis =1), geo_encoded_df], axis =1)
data.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [29]:
# DATA SPLITTING
x = data.drop(["Exited"], axis =1)
y = data["Exited"]

# TRAIN TEST SPLIT
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## STANDARDIZE THE DATA
s = StandardScaler()
x_train = s.fit_transform(x_train)
x_test = s.transform(x_test)



### ANN REGRESSION TRAINED MODEL ###



In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [36]:
## INITIALIZE THE ANN
model = Sequential([
    Dense(64, activation="relu", input_shape=(x_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1)  # OUTPUT LAYER for REGRESSION with  Linear Activation Function
])

## COMPILE THE MODEL

model.compile(optimizer="adam", loss="mean_absolute_error",metrics=['mae'] )
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 


 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [37]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime 

# set up tensorboard callback
log_dir = "reg_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [38]:
# set up early stopping callback
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)


In [39]:
# TRAIN THE MODEL
history = model.fit(x_train,
                    y_train,
                    validation_data=(x_test, y_test),
                    epochs=70,
                    callbacks=[tensorboard_callback, early_stopping_callback])


Epoch 1/70
250/250 [==============================] - 1s 2ms/step - loss: 69737.7734 - mae: 69737.7734 - val_loss: 66091.3281 - val_mae: 66091.3281
Epoch 2/70
250/250 [==============================] - 0s 2ms/step - loss: 63020.8477 - mae: 63020.8477 - val_loss: 57142.7656 - val_mae: 57142.7656
Epoch 3/70
250/250 [==============================] - 0s 1ms/step - loss: 54281.9141 - mae: 54281.9141 - val_loss: 51361.7188 - val_mae: 51361.7188
Epoch 4/70
250/250 [==============================] - 0s 2ms/step - loss: 51782.2773 - mae: 51782.2773 - val_loss: 51053.0391 - val_mae: 51053.0391
Epoch 5/70
250/250 [==============================] - 0s 1ms/step - loss: 51748.7070 - mae: 51748.7070 - val_loss: 51292.3320 - val_mae: 51292.3320
Epoch 6/70
250/250 [==============================] - 0s 2ms/step - loss: 51411.7930 - mae: 51411.7930 - val_loss: 50533.8398 - val_mae: 50533.8398
Epoch 7/70
250/250 [==============================] - 0s 1ms/step - loss: 51290.5273 - mae: 51290.5273 - val_los

In [41]:
# EVALUATE THE MODEL
%load_ext tensorboard


In [44]:
%tensorboard --logdir reg_logs/fit

Reusing TensorBoard on port 6006 (pid 11072), started 0:00:32 ago. (Use '!kill 11072' to kill it.)

In [46]:
## Evaluate the model on test data
test_loss, test_mae = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_mae}")


63/63 [==============================] - 0s 992us/step - loss: 50311.9844 - mae: 50311.9844
Test Loss: 50311.984375


In [47]:
model.save("salary_regression_model.h5")


d:\ANN_Banking_Prediction\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
